# Pynq HDMI Demo

This notebook covers a use case of the pynq project for software video processing demos using HDMI.

The goal is to capture a monitor using the HDMI in convert it to grayscale and perform an algorithm on the current frame. The result will be displayed on another monitor using HDMI out. We will also add triggers for pushbuttons to interact with the demo.

## Loading base overlay

The base Overlay is a bitstream given by pynq usually already present in the Pynq-Z2. It allows us to use peripherals on the board without the need to go through the Vivado design process. In our case, we will use HDMI (in and out) and GPIOS for the buttons.

see : https://pynq.readthedocs.io/en/v2.7.0/pynq_overlays/pynqz2/pynqz2_base_overlay.html

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

## Initialization of HDMI in and out

**HDMI cables must be disconnected before running the cell below otherwise the board may crash**

In [ ]:
from pynq.lib.video import *    # imports constant to configure the hdmi (PIXEL_GRAY)

# creating aliases for easier use
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# configure both hdmi to use pixels in grayscale (8 bit = 1 pixel instead of 24 for RGB)
hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode,PIXEL_GRAY)

# the mode attribute contains informations about 
print(hdmi_in.mode, hdmi_out.mode)

cacheable_frames is a boolean that controls whether frames should be stored in cacheable or non-cacheable memory
by default, it is True but setting it to False may increase performance but software librairies may be SLOWER or NOT WORK.

see : https://github.com/Xilinx/PYNQ/blob/master/boards/Pynq-Z1/base/notebooks/video/hdmi_introduction.ipynb


In [ ]:
hdmi_out.cacheable_frames = False
hdmi_in.cacheable_frames = False

# starts hdmi peripherals and and waits for the HMDI cables to be CONNECTED to the board
hdmi_in.start()
hdmi_out.start()

Now, we are going to import the library "cv2" which is providing us an optimized threshold algorithm and a way to add text to an image. The library "numpy" is used to process homogeneous arrays in python and the library "time" can be used to calculate the number of IPS. We also need to import the function "allocate" from pynq. This function allows us to use contiguous memory in python using a numpy like API.

In [ ]:
import cv2
import numpy as np
from pynq import allocate
import time

We set the variable numframes to 1000 which is enough for us to see the results we want. The buffer for an image needs to be allocated with the right size (screen height * screen width * (bit per pixel / 8)).

There is one concerning issue caused by the hardware limitiation of Pynq-Z2. We are not supposed to enter an image with a resolution higher than 720p when using HDMI as it does not meet the official requirement. It causes performance issues on the output monitor. We tried to reduce the frame size of hdmi in down to 800x600 but the outputting (hdmi out) frame size stays unchanged (720p) and we could not figure out the exact reason. We are using grayscale images allowing us to have the data type of each pixel to go from uint32 (24 bits in RGB) to uint8. It makes the image processing simpler but slows down the pixel unpacking and packing.

In [ ]:
numframes = 1000

data_size = (hdmi_in.mode.height* hdmi_in.mode.width)

grayscale  = allocate(shape=(data_size,), dtype=np.uint8)
output_buffer = allocate(shape=(data_size,), dtype=np.uint8)

start = time.time()

In each loop, we read the frame from the input into an array. A new output frame with the right resolution and bit per pixel size is requested from the ouput hdmi class using the method `newframe()`. The default output should be gray scale images without any other process.

One of the numerous adventages of using python at the PS end is that users do not need to be concerned about for example the addresses of the registries controlling buttons on the card.  When the button 0 is pressed and hold, the frame grayscale will be processed by our own very simple algorithm using numpy (we set the threshold as 127). Likewise, when the button 1 is pressed and hold, PS will execute the function `cv2.threshold(...)`.

At last, by using `reshape()`, the output frames takes the same shape as before the `ravel()`. (At this point, as we are using a laptoop screen as hdmi in, the frame aspect ratio can also influence the output result. Not any aspect ratio (like 16:9) is fully supported by pynq and it can lead to an image shift or displacement).

With timestamps that we defined("start" and "end"), we can also count the fps of output and add it to the output screen using cv2. 

It is also necessary to free the buffers that we created when we get out of the loop but it should be freed by python when going out of the scope. It still is a good practice in case of a crash or an interruption.

In [ ]:
for i in range(numframes):
    inframe = hdmi_in.readframe()
    grayscale[:] = inframe.ravel() # flatten 2d array to 1d
    outframe = hdmi_out.newframe()
    
    mode = "grayscale" 
    
    if base.buttons[0].read(): # checking if the button 0 is being pressed
        # numpy threshold        
        grayscale[grayscale<127] = 0
        grayscale[grayscale>127] = 255
        mode = "threshold SW numpy"    
        
    if base.buttons[1].read():
        # cv2 standard binary threshold
        (T, grayscale_cv2) = cv2.threshold(grayscale, 127, 255, cv2.THRESH_BINARY)
        grayscale[:] = grayscale_cv2.reshape(grayscale.shape)
        mode = "threshold SW cv2"
        
    outframe[:] = grayscale.reshape(*inframe.shape)
    end = time.time()

    # writing current mode and fps on the output screen
    cv2.putText(outframe, text="fps :"+str(int(1 / (end - start))), org=(200,200),fontFace=3, fontScale=3, color=(150,150,150), thickness=5)
    cv2.putText(outframe, text="mode :"+mode, org=(200,350),fontFace=3, fontScale=3, color=(150,150,150), thickness=5)
    
    start = time.time()
    inframe.freebuffer()
    hdmi_out.writeframe(outframe)
    
grayscale.freebuffer()
output_buffer.freebuffer()
outframe.freebuffer()

For the record, we came up with a result of 16-18 fps for the mode "grayscale", 3-4 fps for the mode "threshold SW numpy" and finally 6-8 fps for the mode "threshold SW cv2" for 720p


## Cleanup

This cell is used to clean and close the hdmi peripherals at the end of the demo or to reset the demo in case of an issue on the PL side.

In [ ]:
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out